In [1]:
import capnp
import codecs
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import copy
import statistics

capnp.remove_import_hook()
interface_capnp = capnp.load("../cpp-tool/capnproto/interface.capnp", imports=["/usr/local/include"])

kj/filesystem-disk-unix.c++:1690: warning: PWD environment variable doesn't match current directory; pwd = /root/.vscode-server/bin/e8a3071ea4344d9d48ef8a4df2c097372b0c5161


In [2]:
client = capnp.TwoPartyClient("cpp-tool:21654")
processDataUpdates = client.bootstrap().cast_as(interface_capnp.ProcessDataUpdates)

# Load models

In [3]:
!capnp convert json:binary ../cpp-tool/capnproto/interface.capnp RuleModel < intervalRuleModel.json > /tmp/intervalRuleModel.bin
with open("/tmp/intervalRuleModel.bin", "rb") as file:
    model = interface_capnp.RuleModel.read(file)
print(processDataUpdates.modelToString(model).wait().string)

IF flag = b THEN DELTA num IN [20.000000, 20.100000]



In [4]:
!capnp convert json:binary ../cpp-tool/capnproto/interface.capnp RuleModel < pointRuleModel.json > /tmp/pointRuleModel.bin
with open("/tmp/pointRuleModel.bin", "rb") as file:
    wrongModel = interface_capnp.RuleModel.read(file)
print(processDataUpdates.modelToString(wrongModel).wait().string)

IF flag = b THEN num = num_prev + 20.000000



# Investigate score of both models

In [5]:
for numTraces in [1000]:
    print("Number of traces", numTraces)

    eventLogPromise = processDataUpdates.generateEventLog(model=model, numberOfTraces=numTraces, traceLength=20)
    resolvedEventLogPromise = eventLogPromise.wait()

    scorePromise = processDataUpdates.calculateMdlScore(log=resolvedEventLogPromise.log, model=model)
    score = scorePromise.wait().result
    print("Interval model score:", score)

    scorePromise = processDataUpdates.calculateMdlScore(log=resolvedEventLogPromise.log, model=wrongModel)
    score = scorePromise.wait().result
    print("Point model score:", score)

Number of traces 1000
Interval model score: 158983.65063390826
Point model score: 163434.44665002223


Since point rules only match at perfect precision (here 0.1), they always cause larger encoding costs than interval rules, if the data was generated from an interval. Even for intervals that only span the range of the precision, a point rule results in more bits for encoding the values.